# Example: Dynamic Delta Neutral Hedging to Compensate for Price Fluctuations
[Delta hedging](https://en.wikipedia.org/wiki/Delta_neutral) is a strategy used in options trading to minimize the risk associated with price movements in the underlying asset. This approach involves using other options contracts or shares of the underlying asset to achieve a delta-neutral position, i.e., eliminating the directional bias of the position.
* __Scenario 1__: You are long a `TSLA` put option (thus, you are short the shares) and are concerned about how the price movement of `TSLA` shares influences the overall risk of your position. Toward this risk, let's explore constructing a delta-neutral hedge.
* __Scenario 2__: You are short a `TSLA` call option (thus, you are also short the shares) and are concerned about how the price movement of `TSLA` shares influences the overall risk of your position. Toward this risk, let's explore constructing a delta-neutral hedge.

Both scenarios (s) 1 and 2 make the same directional assumption, namely, that `TSLA` shares will decline in price, i.e., you are `bearish` on `TSLA.` Let's explore how delta-neutral hedging would work for these scenarios.

## Learning objectives
Fill me in.

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Manifest.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
Precompiling project...
  ✓ VLQuantitativeFinancePackage
  1 dependency successfully precompiled in 3 seconds. 282 already precompiled.
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4`
 

## Prerequisites
Fill me in

### Constants

In [6]:
Δt = (1/390)*(1/252); # 1-min
number_of_trading_periods = 60; # How many trading periods will we simulate?
T = number_of_trading_periods*Δt; # 1-trading hour
Sₒ = 261.49; # TSLA share price
IVₒ = 0.627397; # IV for K = 210 put with 60-DTE

Fill me in

In [8]:
option_dataset = load(joinpath(_PATH_TO_DATA, "TSLA-today-2023-06-19-exp-2023-08-18-dte-60d.jld2")) |> file -> file["data"];

In [9]:
option_dataset[50]

Dict{String, Any} with 9 entries:
  "day"                => Dict{String, Any}("change"=>0, "high"=>6.7, "change_p…
  "details"            => Dict{String, Any}("contract_type"=>"put", "ticker"=>"…
  "last_quote"         => Dict{String, Any}("midpoint"=>6.125, "ask"=>6.2, "tim…
  "last_trade"         => Dict{String, Any}("timeframe"=>"REAL-TIME", "exchange…
  "underlying_asset"   => Dict{String, Any}("timeframe"=>"REAL-TIME", "ticker"=…
  "greeks"             => Dict{String, Any}("theta"=>-0.126845, "gamma"=>0.0036…
  "open_interest"      => 3898
  "implied_volatility" => 0.627397
  "break_even_price"   => 203.875

One-aggregate data for `TSLA` for 2023

In [11]:
underlying_price_data_df = CSV.read(joinpath(_PATH_TO_DATA, "TSLA-OHLC-1-min-aggregate-2023.csv"), DataFrame);

Compute the vwap, and the log_growth_matrix 

In [13]:
growth_rate_array = vwap(underlying_price_data_df) |> data -> log_growth_matrix(data, Δt = Δt);

Compute (u,d,p) parameters from data

In [15]:
(ū,d̄,p̄) = (RealWorldBinomialProbabilityMeasure())(growth_rate_array; Δt = Δt);

## Task 1: Construct a Binomial Price Tree to Compute Future Share Prices
Fill me in.

In [17]:
price_tree_model = build(MyBinomialEquityPriceTree, (
    u = ū, d = d̄, p = p̄)) |> (x-> populate(x, Sₒ = Sₒ, h = number_of_trading_periods));

In [41]:
L = 60;
price_array = price_tree_model.levels[L] .|> x-> price_tree_model.data[x].price
probability_array = price_tree_model.levels[L] .|> x-> price_tree_model.data[x].probability;

In [43]:
[price_array probability_array]

61×2 Matrix{Float64}:
 272.509  2.50899e-18
 272.132  1.45256e-16
 271.755  4.13468e-15
 271.379  7.71319e-14
 271.004  1.06056e-12
 270.628  1.14614e-11
 270.254  1.01375e-10
 269.88   7.54593e-10
 269.506  4.82374e-9
 269.133  2.68924e-8
 268.76   1.32338e-7
 268.388  5.80425e-7
 268.017  2.28689e-6
   ⋮      
 254.624  1.49337e-7
 254.272  3.17012e-8
 253.92   5.99779e-9
 253.568  1.00165e-9
 253.217  1.45886e-10
 252.867  1.82475e-11
 252.517  1.92077e-12
 252.167  1.65479e-13
 251.818  1.1205e-14
 251.469  5.59231e-16
 251.121  1.82917e-17
 250.774  2.94163e-19

In [45]:
dot(price_array,probability_array)

261.6136331464506

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.